In [31]:
#!/usr/bin/python 
# -*- coding: utf-8 -*-
!pip install bs4==0.0.1
!pip install elasticsearch==7.10.0

**随机睡眠**

In [32]:
def random_sleep():
    from random import randint
    from time import sleep
    sleep_time = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]
    index = randint(0, len(sleep_time)-1)
    sleep(sleep_time[index])

**随机表头**

In [33]:
def random_header():
    from random import randint
    header_list = [
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0) Gecko/20100101 Firefox/6.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
    "Opera/9.80 (Windows NT 6.1; U; zh-cn) Presto/2.9.168 Version/11.50",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 2.0.50727; SLCC2; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; InfoPath.3; .NET4.0C; Tablet PC 2.0; .NET4.0E)",
    "Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; InfoPath.3)",
    "Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; GTB7.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1)",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; ) AppleWebKit/534.12 (KHTML, like Gecko) Maxthon/3.0 Safari/534.12",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; InfoPath.3; .NET4.0C; .NET4.0E)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; InfoPath.3; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.3 (KHTML, like Gecko) Chrome/6.0.472.33 Safari/534.3 SE 2.X MetaSr 1.0",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; InfoPath.3; .NET4.0C; .NET4.0E)",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.41 Safari/535.1 QQBrowser/6.9.11079.201",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; InfoPath.3; .NET4.0C; .NET4.0E) QQBrowser/6.9.11079.201",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)"]
    index = randint(0, len(header_list)-1)
    return {"user-agent":header_list[index]}

**评论内容预处理**

In [34]:
def re_filter(content):
    '''
    使用re工具, 处理冗余信息
    '''
    import re

    pattern1 = re.compile(r'#[^#]+#')
    pattern2 = re.compile(r"[a-zA-Z]")
    return re.sub(pattern2, "", re.sub(pattern1, "", content))

**评论内容预处理**

In [35]:
def get_info(res, title_name="", get_content=True):
    '''
    处理json数据, 获取指定的内容(题目和评论)或其他内容
    
    Parameters:
        res - request请求后, 返回的response
        title_name - 该评论评价的, 是什么热点题目
        get_content - 判断res请求的网站类型

    Returns:
        content - 依据情况, 返回指定内容
    '''
    from bs4 import BeautifulSoup
    from time import sleep

    if res.status_code != 200 or res.json()["ok"] != 1:
        print("ip被封!, 等候3s")
        sleep(3)
        if get_content:
            return -1, 0
        else:
            return []
    
    contents_dict = res.json()["data"]
    if get_content:
        # 判断是否为浏览页
        ## 如果是
        if "max_id" not in contents_dict:
            mid_list = []
            for item in contents_dict["cards"]:
                # 如果mid_list大于3, 就跳出
                if len(mid_list)>=3:
                    break
                # 判断评论类型
                if item["card_type"] != 9:
                    continue
                # 获取mid
                mid_list.append(item["mblog"]["id"])
            return 1, mid_list
        else:  
        # 如果不是
            contents_list = []
            # 获取下一页
            max_id = contents_dict["max_id"]

            month_dict = {"Jan":"01", "Feb":"02", "Mar":"03", "Apr":"04", "May":"05", "Jun":"06", "Jul":"07","Aug":"08", "Sep":"09", "Oct":"10","Nov":"11","Dec":"12"}
            for item in contents_dict["data"]:

                # 获取评论内容和时间
                content = item["text"]
                time_list = item["created_at"].split(" ")
                created_time = "{}-{}-{}".format(time_list[-1], month_dict[time_list[1]], time_list[2])

                # 使用beautifulSoup
                bf = BeautifulSoup(content)
                content = ""
                for p in bf.stripped_strings:
                    content += p
                
                # 使用re工具, 将##内容全部清除
                content = re_filter(content)
                if content=="" or "转发" in content:
                    continue
                contents_list.append((title_name, content, created_time))
            return max_id, contents_list
    else:
        ## 获取es库已存在title
        old_title = all_es_search(key="title")

        keyword_list = []
        for item in contents_dict["cards"][0]["card_group"]:
            keyword = str(item["desc"])
            if keyword not in old_title:
                keyword_list.append(keyword)
        return keyword_list

**评论数据推入ES库**

In [36]:
def push_es(content_list):
    '''
    将数据推入云es库中
    '''
    # 考虑列表为空时, 直接跳出
    if len(content_list)==0:
        return
    
    # 列表非空时, 开始推入
    from elasticsearch import Elasticsearch
    from elasticsearch import helpers
    # 开始连接es库(感觉这里可以优化)
    es = Elasticsearch(
        ['es-cn-09k1xdlqf000a7yyn.public.elasticsearch.aliyuncs.com'],
        http_auth=('elastic', 'Zy13720028637'),
        port=9200,
        use_ssl=False
    )
    # 判断数据是否为字典类型
    if type(content_list[0]) != dict:
        names = ["title", "describe", "createData"]
        for i, data in enumerate(content_list):
            temp_dict = {}
            for index, content in enumerate(data):
                temp_dict[names[index]] = content
            content_list[i] = temp_dict
    # 将数据修饰一下
    action = ({
        "_index":"hot_news",
        "_type":"comment",
        "_source":data
    } for data in content_list)
    # 导入数据
    helpers.bulk(es, action, request_timeout=30)

**评论网页内容的抓取**

In [37]:
def web_grap(keyword):
    '''
    网页抓取

    Parameters:
        keyword - 热点新闻

    Returns:
        无
    '''
    import requests
    import pandas as pd
    
    base_url = "https://m.weibo.cn/api/container/getIndex?containerid=100103type%3D1%26q%3D%23{}%23&page_type=searchall&page={}"
    # headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"}
    print("\n抓取热度:{}".format(keyword))

    # 第一步抓取热点号
    while True:
        url = base_url.format(keyword, 0)
        random_sleep()
        res = requests.get(url.format(keyword, 1), headers=random_header())
        status, mid_list = get_info(res)
        # 判断ip是否被封
        if status==-1:
            print("再次申请...")
            continue
        else:
            break
    
    base_url = "https://m.weibo.cn/comments/hotflow?id={}&mid={}&max_id={}"
    num = 0
    cookie = {"Cookie":"_T_WM=31420914776;M_WEIBOCN_PARAMS=luicode%3D20000174 ;WEIBOCN_FROM=1110006030; XSRF-TOKEN=6456fc; MLOGIN=1 ;SUB=_2A25yuiimDeRhGeRO61IS9yvLyT-IHXVuRUjurDV6PUJbkdANLWrjkW1NUIeUSgyof564-cNUpoux4c760jBpCOPX"}
    for i in range(len(mid_list)):
        flag = 0
        max_id = 0
        while True:
            # 正常抓取
            url = base_url.format(mid_list[i], mid_list[i], max_id)
            # print(url)
            random_sleep()
            res = requests.get(url, cookies=cookie, headers=random_header())
            temp, contents_list = get_info(res, keyword)

            # 判断ip是否被封
            if temp ==-1:
                flag += 1
                if flag == 1:
                    print("  跳出此次申请...")
                    break
                print("  再次申请...")
                continue
            flag = 0
            num += len(contents_list)
            max_id = temp

            ## 存储数据到es中
            push_es(contents_list)
            print(" 已抓取{}条评论!!".format(num))

            # 判断是否满足跳出条件
            if max_id == 0:
                break
    print("{}抓取结束!".format(keyword))


**用于抓取微博热点标题**

In [38]:
def to_more_grap():
    '''
    用于获取相关热度的关键词

    Parameters:
        无
    
    Returns:
        content_list - 热点新闻列表
    '''
    import requests
    
    # 准备网站
    # base_url = "https://m.weibo.cn/api/container/getIndex?containerid=231583&page_type=searchall"
    base_url = "https://m.weibo.cn/api/container/getIndex?containerid=106003type%3D25%26t%3D3%26disable_hot%3D1%26filter_type%3Drealtimehot&title=%E5%BE%AE%E5%8D%9A%E7%83%AD%E6%90%9C&extparam=seat%3D1%26pos%3D0_0%26mi_cid%3D100103%26cate%3D10103%26filter_type%3Drealtimehot%26c_type%3D30%26display_time%3D1606374389&luicode=10000011&lfid=231583"
    cookie = {"Cookie":"_T_WM=31420914776;M_WEIBOCN_PARAMS=luicode%3D20000174 ;WEIBOCN_FROM=1110006030; XSRF-TOKEN=6456fc; MLOGIN=1 ;SUB=_2A25yuiimDeRhGeRO61IS9yvLyT-IHXVuRUjurDV6PUJbkdANLWrjkW1NUIeUSgyof564-cNUpoux4c760jBpCOPX"}
    
    ## 准备https头部
    # headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"}
    # ## 睡眠
    random_sleep()
    ## 开始访问
    res = requests.get(base_url, headers=random_header(), cookies=cookie)
    ## 处理json数据, 获取更多问题号
    contents_list = get_info(res, get_content=False)
    return contents_list

# **整合爬虫相关代码, 实现爬取流程**

In [39]:
def set_grap(question_num=1):
    '''
    集成爬虫

    Paramenter:
        question_num - 指定爬取的热点新闻数量
    
    Returns:
        无
    '''
    # 爬取关键词
    keyword_list = to_more_grap()
    web_grap(keyword_list[0])
    question_num -= 1

    i = 1
    while question_num:
        temp_i = i
        while len(keyword_list) <= i:
            keyword_list += to_more_grap()
            keyword_list = sorted(set(keyword_list),key=keyword_list.index)
            temp_i -= 1
            if temp_i == -1:
                print("因热度关键词一致, 故抓取结束!")
                return
        web_grap(keyword_list[i])
        i += 1
        question_num -= 1
    print("抓取任务结束!")

**ES库的搜索函数**

    1. 用于搜索题目

In [40]:
def all_es_search(key="title", search=""):
    '''
    输入某一属性, 返回所有不同值的list

    Parameters:
        key - 指搜索的关键词类型
        search - 指搜索内容

    Returns:
        search_list - 返回相关内容的列表
    '''
    from elasticsearch import Elasticsearch
    # 开始连接es库(感觉这里可以优化)
    es = Elasticsearch(
        ['es-cn-09k1xdlqf000a7yyn.public.elasticsearch.aliyuncs.com'],
        http_auth=('elastic', 'Zy13720028637'),
        port=9200,
        use_ssl=False
    )
    # 开始查询(这里绝对可以优化, 先简单实现)
    if len(search) == 0:
        dsl = {
            "size": 0,
            "query":{
                "match_all":{
                    
                }
            },
            "aggs": {
                "source": {
                    "terms": {
                        "field": "{}.raw".format(key),
                        "size" : 1000
                    }
                }
            }
        }
    else:
        dsl = {
            "size": 0,
            "query":{
                "match":{
                    "title":search
                }
            },
            "aggs": {
                "source": {
                    "terms": {
                        "field": "{}.raw".format(key)
                    }
                }
            }
        }

    result = es.search(index='hot_news', doc_type='comment', body=dsl, timeout="30s", ignore=400)

    if "error" in result:
        print("题目列表, 申请失败! 返回空列表")
        return []
    
    mdata = result.get("aggregations").get("source").get("buckets")
        
    if not mdata:
        return []
    
    search_list = []
    for item in mdata:
        search_list.append(item["key"])
    return search_list

    2. 用于搜索指定题目, 将评论数据返回

In [41]:
def es_search(search_in, stype="match", source=("title", )):
    '''
    根据search_in(指定字符串)查找相关评论, 并返回全部内容

    Parameters:
        search_in - 搜索题目
        stype - 搜索模式
        source - 指定es库返回的属性
    
    Returns:
        mdata - 返回评论数据
    '''
    from elasticsearch import Elasticsearch
    # 开始连接es库(感觉这里可以优化)
    es = Elasticsearch(
        ['es-cn-09k1xdlqf000a7yyn.public.elasticsearch.aliyuncs.com'],
        http_auth=('elastic', 'Zy13720028637'),
        port=9200,
        use_ssl=False
    )
    
    # 开始查询(这里绝对可以优化, 先简单实现)
    dsl = {
        "_source" : list(source),
        'query': {
            stype : {
                'title': search_in
            }
        }
    }

    result = es.search(index='hot_news', doc_type='comment', body=dsl, scroll="30s", size=100, timeout="30s")
    mdata = result.get("hits").get("hits")
        
    if not mdata:
        print('empty')
        return []
    scroll_id = result["_scroll_id"]
    total = result["hits"]["total"]
    for i in range(int(total/100)):
        res = es.scroll(scroll_id=scroll_id,scroll='30s')
        mdata = mdata + res["hits"]["hits"]
    return mdata

**处理从ES库获取下来的数据, 整合成列表**

In [42]:
def get_search_info(content, source=("title",)):
    '''
    先简单一点, 就获取相关问题title
    '''
    data_list = []
    source_list = list(source)
    for data in content:
        source = data["_source"]
        temp_list = []
        for name in source_list:
            temp_list.append(source[name])
        
        # 保证数据维度尽量不多
        if len(temp_list)==1:
            data_list.append(temp_list[0])
        else:
             data_list.append(temp_list)
    return data_list

**标题的打印**

In [43]:
def title_print(content_info):
    '''
    简单实现, 就打印含有title的列表
    '''
    print("  搜索结果:")
    for index, title in enumerate(content_info):
        print("    {}. {}".format(index+1, title))
    if len(content_info)==0:
        print("  未搜索相关结果!")

**将评论数据存储为csv格式**

In [44]:
def save_csv(content_info, names=("date", "describe")):
    '''
    将列表数据转换为csv数据存储
    '''
    import pandas as pd
    content_info = [x for x in content_info if x[1]!=""]
    dataset = pd.DataFrame(content_info, columns=names)
    dataset.to_csv("news.csv", index=None, sep="\t")

# **整合基于ES库搜索的相关代码**

In [58]:
def weibo_search():
    # 输入相关查询
    search_in = input()
    # 从es库查询结果
    print("  " + search_in)
    content = all_es_search(search=search_in)
    # 打印内容
    title_print(content)

    if len(content)==0:
        return

    # 二级执行菜单
    print("\n------二级执行菜单------")
    print("1. 选择指定题目--返回评论数据")
    print("2. 返回一级菜单")
    select_num = int(input())
    if select_num==1:
        print("\n  请选择题目的标题号:")
        title_num = int(input())
        print("    你选择的标题号: {}".format(title_num))
        content = es_search(content[title_num-1], stype="match_phrase", source=("createData", "describe",))
        content_info = get_search_info(content, source=("createData", "describe",))
        # print("数据数量:{}".format(len(content_info)))
        # print("数据元素:{}".format(content_info[0]))
        # save_csv(content_info)
    else:
        return 0

# **结果可视化**

In [46]:
class eval_visual():
    '''
    用于可视化评论分布, 暂定目标饼图和线性图
    '''
    def __init__(self, title, comment):
        self.title = title
        self.comment = comment  # [(date, price), ...]
        self.data_dict = {}
        self.deal_data()
    
    def deal_data(self):
        # 处理数据
        if type(self.comment) != list:
            print("数据类型为:{}\t处理失败!".format(type(self.comment)))
            return
        # 需要整体的正类的数据个数, 负类的数据个数
        if type(self.comment[0][1]) != int:
            self.comment = [[x[0], int(x[1])] for x in self.comment]
        total = len(self.comment)
        total_0_data = [x for x in self.comment if x[1]==0]
        self.data_dict["total_data"] = {}
        self.data_dict["total_data"]["0"] = len(total_0_data)
        total_1_data = [x for x in self.comment if x[1]==1]
        self.data_dict["total_data"]["1"] = len(total_1_data)
        
        # 需要不同时间下, 正类的数据个数, 负类的数据个数
        self.data_dict["date_data"] = {}
        self.date = list(set([x[0] for x in self.comment]))
        self.date.sort()
        ## 假设正类和负类的每天都会有
        self.data_dict["date_data"]["0"] = []
        self.data_dict["date_data"]["1"] = []
        for x in self.date:
            self.data_dict["date_data"]["0"].append(len([y for y in total_0_data if y[0]==x]))
            self.data_dict["date_data"]["1"].append(len([y for y in total_1_data if y[0]==x]))


    def paint(self):
        import matplotlib.pyplot as plt
        plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
        plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
        # 划分子图
        fig, ax = plt.subplots(1,2, figsize=(18, 6))
        fig.suptitle(self.title, fontsize=25)
        # 画饼图
        ax[0].set_title("情感分析", fontsize=16)
        self.paint_pie(ax[0])
        # 画线性图
        ax[1].set_title("时间趋势", fontsize=16)
        self.paint_line(ax[1])
        # 展示
        plt.show()
    
    def paint_pie(self, ax):
        total_0_num = self.data_dict["total_data"]["0"]
        total_1_num = self.data_dict["total_data"]["1"]
        sizes = [total_0_num, total_1_num]
        labels = ["Bad", "Good"]
        # 画图
        ax.pie(sizes, labels=labels, autopct='%.2f%%', shadow=True, startangle=90)
    
    def paint_line(self, ax):
        index = list(range(len(self.date)))
        data_0 = self.data_dict["date_data"]["0"]
        data_1 = self.data_dict["date_data"]["1"]
        ax.plot(index, data_0, label="bad")
        ax.plot(index, data_1, label="good")
        ax.set_xticks(index)
        ax.set_xticklabels(self.date, rotation=45)
        ax.set_xlabel("日期", fontsize=12)
        ax.set_ylabel("数量", fontsize=12)
        ax.legend()

In [61]:
def main():
    while True:
        print("\n------一级执行菜单------")
        print("1. 简单抓取")
        print("2. 相关搜索")
        print("3. 退出程序")
        select_num = int(input())
        if select_num==1:
            set_grap(10)
        elif select_num==2:
            print("\n  请输入内容:")
            weibo_search()
        else:
            print("\n成功退出!")
            break

In [62]:
main()


------一级执行菜单------
1. 简单抓取
2. 相关搜索
3. 退出程序

抓取热度:上海地铁禁止电子设备声音外放
 已抓取20条评论!!
 已抓取40条评论!!
 已抓取60条评论!!
 已抓取78条评论!!
 已抓取97条评论!!
 已抓取116条评论!!
 已抓取134条评论!!
 已抓取151条评论!!
 已抓取171条评论!!
 已抓取189条评论!!
 已抓取207条评论!!
 已抓取226条评论!!
 已抓取244条评论!!
 已抓取263条评论!!
 已抓取283条评论!!
ip被封!, 等候3s
  跳出此次申请...
 已抓取303条评论!!
 已抓取323条评论!!
 已抓取343条评论!!
 已抓取355条评论!!
 已抓取356条评论!!
 已抓取376条评论!!
 已抓取396条评论!!
 已抓取416条评论!!
 已抓取436条评论!!
 已抓取453条评论!!
 已抓取472条评论!!
 已抓取480条评论!!
上海地铁禁止电子设备声音外放抓取结束!

抓取热度:秋冬底妆避雷
 已抓取20条评论!!
 已抓取38条评论!!
 已抓取50条评论!!
 已抓取54条评论!!
 已抓取74条评论!!
 已抓取93条评论!!
 已抓取112条评论!!
 已抓取115条评论!!
ip被封!, 等候3s
  跳出此次申请...
 已抓取135条评论!!
 已抓取153条评论!!
 已抓取173条评论!!
 已抓取193条评论!!
 已抓取213条评论!!
 已抓取233条评论!!
 已抓取252条评论!!
 已抓取256条评论!!
 已抓取257条评论!!
 已抓取258条评论!!
ip被封!, 等候3s
  跳出此次申请...
秋冬底妆避雷抓取结束!

抓取热度:汪峰大女儿侧颜
 已抓取20条评论!!
 已抓取39条评论!!
 已抓取59条评论!!
 已抓取78条评论!!
 已抓取98条评论!!
 已抓取118条评论!!
 已抓取138条评论!!
 已抓取157条评论!!
 已抓取176条评论!!
 已抓取196条评论!!
 已抓取216条评论!!
 已抓取236条评论!!
 已抓取256条评论!!
 已抓取276条评论!!
 已抓取296条评论!!
ip被封!, 等候3s
  跳出此次申请...
 已抓取316条评论!!
 已抓